In [ ]:
using Pkg
pkg"add Luxor"

In [ ]:
using Luxor

# Trivial Example

In [ ]:
function rule0dim(x::Bool)
    !x
end

In [ ]:
function step0dim(x₀::Bool, steps::Int64)
    xs = [x₀]
    for i in 1:steps
        push!(xs, rule0dim(xs[end]))
    end
    xs
end

In [ ]:
res = step0dim(false, 10)
Drawing(50, 120, "out.svg")
for (i, val) in enumerate(res)
    if val
        sethue("grey")
    else
        sethue("lightgrey")
    end
    box(5, 5+i*10, 10, 10, :fill)
end
finish()
preview()

# Wolfram's Experiment

In [ ]:
function inttorule1dim(val::Int64)
    digs = BitArray(digits(val, base=2))
    #pushfirst!(digs, false)
    for i in length(digs):7
        push!(digs, false)
    end
    if digs[1] == true || digs[2] ≠ digs[5] || digs[7] ≠ digs[4]
        #error("$val: is not a legal rule")
    end
    digs
end

In [ ]:
println(digits(50, base=2))
inttorule1dim(50)

In [ ]:
function applyrule1dim(rule::BitArray{1}, bits::BitArray{1})
    val = 1 + bits[3] + 2*bits[2] + 4*bits[1]
    rule[val]
end

In [ ]:
function step1dim(x₀::BitArray{1}, rule::BitArray{1}, steps::Int64)
    xs = [x₀]
    len = length(x₀)
    for i in 1:steps
        x = copy(x₀)
        for j in 2:len-1
            x[j] = applyrule1dim(rule, xs[end][j-1:j+1])
        end
        push!(xs, x)
    end
    xs
end

In [ ]:
x₀ = falses(21)
x₀[11] = true
res = step1dim(x₀, inttorule1dim(50), 9);

In [ ]:
function visualize1dim(res, dim)
    width = dim * (length(res[1]) + 1)
    height = dim * (length(res) + 1)
    Drawing(width, height, "out.svg")
    for (i, arr) in enumerate(res)
        for (j, val) in enumerate(arr)
            if val
                sethue("grey")
            else
                sethue("lightgrey")
            end
            box(j*dim, i*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end 

In [ ]:
visualize1dim(res, 8)

In [ ]:
x₀ = falses(129)
x₀[65] = true
res = step1dim(x₀, inttorule1dim(18), 63);
visualize1dim(res, 4)

In [ ]:
x₀ = falses(201)
x₀[101] = true
res = step1dim(x₀, inttorule1dim(30), 99);
visualize1dim(res, 2)

In [ ]:
x₀ = falses(201)
x₀[101] = true
res = step1dim(x₀, inttorule1dim(161), 99);
visualize1dim(res, 2)

In [ ]:
x₀ = BitArray{1}(undef, 600)
res = step1dim(x₀, inttorule1dim(110), 599);
visualize1dim(res, 0.75)

# Turing State-Machine

| Tape Symbol | State A   | State B   | State C   |
|:-----------:|-----------|-----------|-----------|
| 0           | 1 - R - B | 1 - L - A | 1 - L - B |
| 1           | 1 - L - C | 1 - R - B | 1 - R - H |

In [ ]:
function applyrulebusybeaver(state, read)
    if state == 'A' && read == 0
        return 1, 'R', 'B'
    elseif state == 'A' && read == 1
        return 1, 'L', 'C'
    elseif state == 'B' && read == 0
        return 1, 'L', 'A'
    elseif state == 'B' && read == 1
        return 1, 'R', 'B'
    elseif state == 'C' && read == 0
        return 1, 'L', 'B'
    elseif state == 'C' && read == 1
        return 1, 'R', 'H'
    end
end

In [ ]:
mutable struct Turing
    tape :: Array{Int64}
    position :: Int64
    state :: Char
end

In [ ]:
function Base.show(io::IO, turing::Turing)
    print(io, turing.position, " - ", turing.state, ": ", turing.tape)
end

In [ ]:
function stepturing(turing, applyrule)
    if turing.state == 'H'
        error("Machine has stopped!")
    end
    read = turing.tape[turing.position]
    (write, dir, turing.state) = applyrule(turing.state, read)
    turing.tape[turing.position] = write
    if dir == 'L'
        if turing.position == length(turing.tape)
            push!(turing.tape, false)
        end
        turing.position += 1
    else
        if turing.position == 1
            pushfirst!(turing.tape, false)
        else
            turing.position -= 1
        end
    end
    nothing
end

In [ ]:
turing = Turing(zeros(Int64, 11), 6, 'A')
println(turing)
while true
    stepturing(turing, applyrulebusybeaver)
    println(turing)
end

# Game of Life

In [ ]:
function applyrulegameoflife(bits::BitArray{2})
    (nr_y, nr_x) = size(bits)
    out = falses(nr_y, nr_x)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            if bits[y, x]
                if 2 ≤ count(v->v, bits[y-1:y+1,x-1:x+1]) - 1 ≤ 3
                    out[y, x] = true
                end
            else
                if count(v->v, bits[y-1:y+1,x-1:x+1]) == 3
                    out[y, x] = true
                end
            end
        end
    end
    out
end

In [ ]:
function visualize2dim(bits::BitArray{2}, dim)
    (nr_y, nr_x) = size(bits)
    width = dim * (nr_x - 1)
    height = dim * (nr_y - 1)
    Drawing(width, height, "out.svg")
    for (j, y) in enumerate(2:nr_y-1)
        for (i, x) in enumerate(2:nr_x-1)
            if bits[y, x]
                sethue("grey")
            else
                sethue("lightgrey")
            end
            box(i*dim, j*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end

In [ ]:
beehive = falses(5, 6)
beehive[2,3:4] = [true, true]
beehive[3,2] = true
beehive[3,5] = true
beehive[4,3:4] = [true, true]
visualize2dim(beehive, 32)

In [ ]:
beehive = applyrulegameoflife(beehive)
visualize2dim(beehive, 32)

In [ ]:
toad = falses(6, 6)
toad[3,3:5] = [true, true, true]
toad[4,2:4] = [true, true, true]
visualize2dim(toad, 32)

In [ ]:
toad = applyrulegameoflife(toad)
visualize2dim(toad, 32)

In [ ]:
r_pentomino = falses(66, 66)
r_pentomino[28,28:29] = [true, true]
r_pentomino[29,27:28] = [true, true]
r_pentomino[30,28] = true
visualize2dim(r_pentomino, 4)

In [ ]:
r_pentomino = applyrulegameoflife(r_pentomino)
visualize2dim(r_pentomino, 4)

# Diffusion

In [ ]:
function applydiffusion(array::Array{Float64, 2}, r::Float64=0.1)
    nr_y, nr_x = size(array)
    out = deepcopy(array)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            c = array[y-1, x] + array[y, x-1] + array[y, x+1] + array[y+1, x] - 4*array[y, x]
            out[y, x] += r*c
        end
    end
    out
end

In [ ]:
function visualizearray(array::Array{Float64, 2}, dim)
    (nr_y, nr_x) = size(array)
    width = dim * (nr_x - 1)
    height = dim * (nr_y - 1)
    Drawing(width, height, "out.svg")
    for (j, y) in enumerate(2:nr_y-1)
        for (i, x) in enumerate(2:nr_x-1)
            sethue(setgray(0.8*(1-array[y, x])+0.1))
            box(i*dim, j*dim, dim, dim, :fill)
        end
     end
     finish()
     preview()
end

In [ ]:
array = zeros(Float64, 11, 11)
array[5:7, 5:7] = ones(Float64, 3, 3)
visualizearray(array, 16)

In [ ]:
for i in 1: 10; array = applydiffusion(array); end
visualizearray(array, 16)

# Reaction-Diffusion

In [ ]:
function applyreactiondiffusion(a::Array{Float64, 2}, b::Array{Float64, 2}, ra::Float64=0.5, rb=Float64=0.25, 
        f::Float64=0.055, k = Float64=0.062)
    nr_y, nr_x = size(a)
    a_out = deepcopy(a)
    b_out = deepcopy(b)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            reaction = a[y, x] * b[y, x]^2
            ca = 0.25*(a[y-1, x] + a[y, x-1] + a[y, x+1] + a[y+1, x]) - a[y, x]
            cb = 0.25*(b[y-1, x] + b[y, x-1] + b[y, x+1] + b[y+1, x]) - b[y, x]
            a_out[y, x] += ra*ca - reaction + f * (1 - a[y, x])
            b_out[y, x] += rb*cb + reaction - (f+k) * b[y, x]
        end
    end
    a_out, b_out
end

In [ ]:
a = ones(Float64, 258, 258)
b = rand(Float64, 258, 258)*0.1
b[129-12:129+12, 129-12:129+12] += ones(Float64, 25, 25)*0.1
visualizearray(b, 2)

In [ ]:
# f = 0.035, 0.055, 0.039 k = 0.057, 0.062, 0.065
for i in 1:1000
    global a, b
    a, b = applyreactiondiffusion(a, b, 0.5, 0.25, 0.039, 0.065)
end
visualizearray(b, 2)

# Percolation

In [ ]:
function applypercolation(array::Array{Float64, 2})
    nr_y, nr_x = size(array)
    out = deepcopy(array)
    for y in 2:nr_y-1
        for x in 2:nr_x-1
            if out[y, x] > 0.0
                c = array[y-1, x] + array[y, x-1] + array[y, x+1] + array[y+1, x]
                if c ≥ 0.5
                    out[y, x] = 0.5
                end
            end
        end
    end
    out
end

In [ ]:
array = zeros(Float64, 102, 102)
array[2, 2:101] = ones(Float64, 100)*0.5
array[3:101, 2:101] = rand(Float64, 99, 100)
for y in 3:101
    for x in 2:101
        if array[y, x] < 0.5
            array[y, x] = 0.1
        else
            array[y, x] = 0.0
        end
    end
end
visualizearray(array, 4)

In [ ]:
for i in 1:100
    array = applypercolation(array)
end
visualizearray(array, 4)